In [170]:
from json import loads
import numpy as np
import pandas as pd
from pyomo.environ import *

costs = None
with open("costs.json") as f:
    costs = list(loads(f.read()).items())

print(f"Parts in DB: {len(costs)}")

Parts in DB: 108


In [231]:
def prop_vector(prop_name):
    return np.array([data.get(prop_name, 0) for _, data in costs])

names = [n for n, _ in costs]
hitboxes = [np.array(data.get('hitbox', np.ones([data.get('width'),data.get('height')]))) for _, data in costs]
area = np.array([np.count_nonzero(hb) for hb in hitboxes])

data = pd.DataFrame({
    'part': names,
    'cost': area * prop_vector('cost'),
    'energy': area * prop_vector('energy'),
    'fuel': area * prop_vector('fuel'),
    'firepower': area * prop_vector('firepower'),
    'passenger': area * prop_vector('quarters'),
    'command': area * prop_vector('command'),
    'cargo': area * prop_vector('cargo'),
})

data.set_index('part', inplace=True)
data

,cost,energy,fuel,firepower,passenger,command,cargo
part,,,,,,,
antenna_1x2,100.0,0.0,0,0,0,0,0
antenna_1x4,400.0,0.0,0,0,0,0,0
antenna_1x5,300.0,0.0,0,0,0,0,0
bunk_1x1,20.0,0.0,0,0,1,0,0
bunk_2x1,40.0,0.0,0,0,2,0,0
...,...,...,...,...,...,...,...
tank_pod_3x3,315.0,0.0,9,0,0,0,0
tractorbeam_2x3,120.0,-8.0,0,0,0,0,0
turbine_2x2,900.0,40.0,0,0,0,0,0


In [233]:
model = ConcreteModel()
model.x = Var(names, domain=NonNegativeIntegers)
model.total_cost = Objective(
    expr = sum(cost[i]*model.x[i] for i in names),
    sense = minimize
)
model.fuel = Constraint(
    expr = sum(data.loc[i].fuel * model.x[i] for i in names) >= 12
)
model.energy = Constraint(
    expr = sum(data.loc[i].energy * model.x[i] for i in names) >= 10
)
model.firepower = Constraint(
    expr = sum(data.loc[i].firepower * model.x[i] for i in names) >= 53
)
model.passenger = Constraint(
    expr = sum(data.loc[i].passenger * model.x[i] for i in names) >= 10
)
model.command = Constraint(
    expr = sum(data.loc[i].command * model.x[i] for i in names) >= 10
)
model.cargo = Constraint(
    expr = sum(data.loc[i].cargo * model.x[i] for i in names) >= 10
)

solver=SolverFactory('glpk', executable="/usr/local/bin/glpsol")
results = solver.solve(model)
print(results)
# model.display()


res = []
for i in model.x:
    if(model.x[i].value > 0):
        res.append([str(model.x[i])[2:-1], int(model.x[i].value)])
        
df = pd.DataFrame(res, columns=['Part','Qty'])
df.style.hide_index()


Problem: 
- Name: unknown
  Lower bound: 4635.0
  Upper bound: 4635.0
  Number of objectives: 1
  Number of constraints: 7
  Number of variables: 109
  Number of nonzeros: 68
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 191
      Number of created subproblems: 191
  Error rc: 0
  Time: 0.015452861785888672
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



Part,Qty
bunk_1x1,10
cannon_2x1,24
cannon_5x1,1
cargo_5x2,1
cockpit_1x2,1
cockpit_1x4,2
generator_1x1,1
generator_2x2,1
solar_panel_3x1,1
tank_1x1,4
